In [4]:
!pip install safetensors huggingface_hub safetensors transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#!huggingface-cli login

In [6]:
#!git config --global credential.helper store

In [7]:
import torch
from safetensors import safe_open
from safetensors.torch import save_file
from transformers import AutoTokenizer, AutoModelForCausalLM

In [8]:
tokenizer = AutoTokenizer.from_pretrained("dfurman/llama-7b-fp16")

model = AutoModelForCausalLM.from_pretrained("dfurman/llama-7b-fp16",
                                             torch_dtype=torch.float16)
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [9]:
!mkdir llama_7b
model.save_pretrained('/content/llama_7b')
tokenizer.save_pretrained('/content/llama_7b')

('/content/llama_7b/tokenizer_config.json',
 '/content/llama_7b/special_tokens_map.json',
 '/content/llama_7b/tokenizer.json')

In [10]:
import glob
glob.glob('/content/llama_7b/*.bin')

['/content/llama_7b/pytorch_model-00001-of-00002.bin',
 '/content/llama_7b/pytorch_model-00002-of-00002.bin']

In [11]:
# iterate over each bin file in model folder
# expected filenames: pytorch_model-0000x-of-0000y.bin

bins = glob.glob('/content/llama_7b/*.bin')

for tensor_path in bins:
    tensor = torch.load(tensor_path)
    new_name = tensor_path.split('/')[-1].split('pytorch_')[-1].split('.bin')[0]
    print(tensor_path)
    print(new_name)
    save_file(tensor, f"/content/llama_7b/{new_name}.safetensors")


    tensors = {}
    with safe_open(f"/content/llama_7b/{new_name}.safetensors", framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)
    print(tensors.keys())
    print("\n")


/content/llama_7b/pytorch_model-00001-of-00002.bin
model-00001-of-00002
dict_keys(['model.embed_tokens.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.10.input_layernorm.weight', 'm

In [12]:
model_name = "/content/llama_7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

print(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [14]:

inputs = tokenizer("Paris is the capital of", return_tensors="pt", return_token_type_ids=False)

# Example 1: Print the scores for each token generated with Greedy Search
outputs = model.generate(**inputs, max_new_tokens=100)
tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'Paris is the capital of France and the most populous city in the country. It is situated on the river Seine, in the north of the country, at the heart of the Île-de-France region. The city of Paris has an estimated population of 2,241,346 (January 2014) and the Paris metropolitan area has a population of 12,005,077 (January 2014), or'

In [ ]:
# optionally push to hub

# model.push_to_hub("dfurman/llama-7b-fp16", use_auth_token=True)
# tokenizer.push_to_hub("dfurman/llama-7b-fp16", use_auth_token=True)